In [ ]:
import requests
from langchain.tools import tool
import os

#from get_api_keys import langsmithapi_key, openapi_key 
os.environ["LANGCHAIN_TRACING_V2"] = "true"
#
os.environ["LANGSMITH_API_KEY"] = ''
os.environ["LANGSMITH_PROJECT"]="Weather_tool_testing"
#os.environ["OPENAI_API_KEY"] = openapi_key




In [ ]:

#from get_api_keys import weatherapi_key
#
os.environ["OPENWEATHER_KEY"] = ''

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=""
)

In [23]:
result = llm.invoke("What is the capital of india ?")
print(result)

content='The capital of India is New Delhi.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-Caba14SfWm7JrAGygVvEHKbYI7hEW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--19d6e2e4-1f01-4356-bcc9-cb660bffec54-0' usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [24]:
import os
import requests
from langchain.tools import tool


@tool
def get_weather(city: str) -> str:
    """Fetch real-time weather for a given city using OpenWeatherMap API."""
    
    api_key = os.getenv("OPENWEATHER_KEY")
    url = (
        f"https://api.openweathermap.org/data/2.5/weather?"
        f"q={city}&appid={api_key}&units=metric"
    )
    
    response = requests.get(url).json()

    if "weather" not in response:
        return f"Error: Could not fetch weather for {city}. {response.get('message', '')}"

    desc = response["weather"][0]["description"]
    temp = response["main"]["temp"]
    humidity = response["main"]["humidity"]

    return f"Weather in {city}: {desc}, {temp}°C, humidity {humidity}%"


In [25]:
llm_with_tools = llm.bind_tools([get_weather])

In [26]:

from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph, START, END, MessagesState

In [27]:
def call_llm(state:MessagesState):
    """LLM decides whether to call tool or answer directly"""
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

In [28]:
workflow = StateGraph(MessagesState)

workflow.add_node("llm", call_llm)
workflow.add_node("tools",ToolNode([get_weather]))

workflow.add_conditional_edges("llm", tools_condition, {"tools":"tools", END: END})
workflow.add_edge("tools","llm")
workflow.set_entry_point("llm")

graph = workflow.compile()

In [29]:
response = graph.invoke({"messages": [{"role":"user", "content":"What is the weather of Chennai"}]})
print(response['messages'][-1].content)

The weather in Chennai is currently misty, with a temperature of 30.91°C and humidity at 73%.
